<a href="https://colab.research.google.com/github/Sindhuhar/deep_learning/blob/main/backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backpropagation from scratch with Python


### Import Packages

In [2]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import datasets
import numpy as np

### Implementing Backpropagation with Python

In [3]:
class NeuralNetwork:
	def __init__(self, layers, alpha=0.1):
		# initialize the list of weights matrices, then store the
		# network architecture and learning rate
		self.W = []
		self.layers = layers
		self.alpha = alpha

		# start looping from the index of the first layer but
		# stop before we reach the last two layers
		for i in np.arange(0, len(layers) - 2):
			# randomly initialize a weight matrix connecting the
			# number of nodes in each respective layer together,
			# adding an extra node for the bias
			w = np.random.randn(layers[i] + 1, layers[i + 1] + 1)
			self.W.append(w / np.sqrt(layers[i]))

		# the last two layers are a special case where the input
		# connections need a bias term but the output does not
		w = np.random.randn(layers[-2] + 1, layers[-1])
		self.W.append(w / np.sqrt(layers[-2]))

	def __repr__(self):
		# construct and return a string that represents the network
		# architecture
		return "NeuralNetwork: {}".format(
			"-".join(str(l) for l in self.layers))

	def sigmoid(self, x):
		# compute and return the sigmoid activation value for a
		# given input value
		return 1.0 / (1 + np.exp(-x))

	def sigmoid_deriv(self, x):
		# compute the derivative of the sigmoid function ASSUMING
		# that `x` has already been passed through the `sigmoid`
		# function
		return x * (1 - x)

	def fit(self, X, y, epochs=1000, displayUpdate=100):
		# insert a column of 1's as the last entry in the feature
		# matrix -- this little trick allows us to treat the bias
		# as a trainable parameter within the weight matrix
		X = np.c_[X, np.ones((X.shape[0]))]

		# loop over the desired number of epochs
		for epoch in np.arange(0, epochs):
			# loop over each individual data point and train
			# our network on it
			for (x, target) in zip(X, y):
				self.fit_partial(x, target)

			# check to see if we should display a training update
			if epoch == 0 or (epoch + 1) % displayUpdate == 0:
				loss = self.calculate_loss(X, y)
				print("[INFO] epoch={}, loss={:.7f}".format(
					epoch + 1, loss))

	def fit_partial(self, x, y):
		# construct our list of output activations for each layer
		# as our data point flows through the network; the first
		# activation is a special case -- it's just the input
		# feature vector itself
		A = [np.atleast_2d(x)]

		# FEEDFORWARD:
		# loop over the layers in the network
		for layer in np.arange(0, len(self.W)):
			# feedforward the activation at the current layer by
			# taking the dot product between the activation and
			# the weight matrix -- this is called the "net input"
			# to the current layer
			net = A[layer].dot(self.W[layer])

			# computing the "net output" is simply applying our
			# non-linear activation function to the net input
			out = self.sigmoid(net)

			# once we have the net output, add it to our list of
			# activations
			A.append(out)

		# BACKPROPAGATION
		# the first phase of backpropagation is to compute the
		# difference between our *prediction* (the final output
		# activation in the activations list) and the true target
		# value
		error = A[-1] - y

		# from here, we need to apply the chain rule and build our
		# list of deltas `D`; the first entry in the deltas is
		# simply the error of the output layer times the derivative
		# of our activation function for the output value
		D = [error * self.sigmoid_deriv(A[-1])]

		# once you understand the chain rule it becomes super easy
		# to implement with a `for` loop -- simply loop over the
		# layers in reverse order (ignoring the last two since we
		# already have taken them into account)
		for layer in np.arange(len(A) - 2, 0, -1):
			# the delta for the current layer is equal to the delta
			# of the *previous layer* dotted with the weight matrix
			# of the current layer, followed by multiplying the delta
			# by the derivative of the non-linear activation function
			# for the activations of the current layer
			delta = D[-1].dot(self.W[layer].T)
			delta = delta * self.sigmoid_deriv(A[layer])
			D.append(delta)

		# since we looped over our layers in reverse order we need to
		# reverse the deltas
		D = D[::-1]

		# WEIGHT UPDATE PHASE
		# loop over the layers
		for layer in np.arange(0, len(self.W)):
			# update our weights by taking the dot product of the layer
			# activations with their respective deltas, then multiplying
			# this value by some small learning rate and adding to our
			# weight matrix -- this is where the actual "learning" takes
			# place
			self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

	def predict(self, X, addBias=True):
		# initialize the output prediction as the input features -- this
		# value will be (forward) propagated through the network to
		# obtain the final prediction
		p = np.atleast_2d(X)

		# check to see if the bias column should be added
		if addBias:
			# insert a column of 1's as the last entry in the feature
			# matrix (bias)
			p = np.c_[p, np.ones((p.shape[0]))]

		# loop over our layers in the network
		for layer in np.arange(0, len(self.W)):
			# computing the output prediction is as simple as taking
			# the dot product between the current activation value `p`
			# and the weight matrix associated with the current layer,
			# then passing this value through a non-linear activation
			# function
			p = self.sigmoid(np.dot(p, self.W[layer]))

		# return the predicted value
		return p

	def calculate_loss(self, X, targets):
		# make predictions for the input data points then compute
		# the loss
		targets = np.atleast_2d(targets)
		predictions = self.predict(X, addBias=False)
		loss = 0.5 * np.sum((predictions - targets) ** 2)

		# return the loss
		return loss

### Backpropagation with Python Example #1: Bitwise XOR


In [4]:
# construct the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

In [5]:
# define our 2-2-1 neural network and train it
nn = NeuralNetwork([2, 2, 1], alpha=0.5)
nn.fit(X, y, epochs=20000)

[INFO] epoch=1, loss=0.5167123
[INFO] epoch=100, loss=0.4854492
[INFO] epoch=200, loss=0.4619514
[INFO] epoch=300, loss=0.4054542
[INFO] epoch=400, loss=0.3136202
[INFO] epoch=500, loss=0.1951772
[INFO] epoch=600, loss=0.1086938
[INFO] epoch=700, loss=0.0661831
[INFO] epoch=800, loss=0.0450210
[INFO] epoch=900, loss=0.0332361
[INFO] epoch=1000, loss=0.0259775
[INFO] epoch=1100, loss=0.0211472
[INFO] epoch=1200, loss=0.0177390
[INFO] epoch=1300, loss=0.0152232
[INFO] epoch=1400, loss=0.0132993
[INFO] epoch=1500, loss=0.0117857
[INFO] epoch=1600, loss=0.0105668
[INFO] epoch=1700, loss=0.0095662
[INFO] epoch=1800, loss=0.0087313
[INFO] epoch=1900, loss=0.0080249
[INFO] epoch=2000, loss=0.0074202
[INFO] epoch=2100, loss=0.0068970
[INFO] epoch=2200, loss=0.0064402
[INFO] epoch=2300, loss=0.0060382
[INFO] epoch=2400, loss=0.0056818
[INFO] epoch=2500, loss=0.0053638
[INFO] epoch=2600, loss=0.0050785
[INFO] epoch=2700, loss=0.0048211
[INFO] epoch=2800, loss=0.0045877
[INFO] epoch=2900, loss=0.

In [6]:
# now that our network is trained, loop over the XOR data points
for (x, target) in zip(X, y):
	# make a prediction on the data point and display the result
	# to our console
	pred = nn.predict(x)[0][0]
	step = 1 if pred > 0.5 else 0
	print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
		x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.0043, step=0
[INFO] data=[0 1], ground-truth=1, pred=0.9835, step=1
[INFO] data=[1 0], ground-truth=1, pred=0.9838, step=1
[INFO] data=[1 1], ground-truth=0, pred=0.0194, step=0


In [7]:
# define our 2-1 neural network and train it
nn = NeuralNetwork([2, 1], alpha=0.5)
nn.fit(X, y, epochs=20000)

[INFO] epoch=1, loss=0.7793258
[INFO] epoch=100, loss=0.5004587
[INFO] epoch=200, loss=0.5006453
[INFO] epoch=300, loss=0.5007688
[INFO] epoch=400, loss=0.5007899
[INFO] epoch=500, loss=0.5007932
[INFO] epoch=600, loss=0.5007937
[INFO] epoch=700, loss=0.5007938
[INFO] epoch=800, loss=0.5007938
[INFO] epoch=900, loss=0.5007938
[INFO] epoch=1000, loss=0.5007938
[INFO] epoch=1100, loss=0.5007938
[INFO] epoch=1200, loss=0.5007938
[INFO] epoch=1300, loss=0.5007938
[INFO] epoch=1400, loss=0.5007938
[INFO] epoch=1500, loss=0.5007938
[INFO] epoch=1600, loss=0.5007938
[INFO] epoch=1700, loss=0.5007938
[INFO] epoch=1800, loss=0.5007938
[INFO] epoch=1900, loss=0.5007938
[INFO] epoch=2000, loss=0.5007938
[INFO] epoch=2100, loss=0.5007938
[INFO] epoch=2200, loss=0.5007938
[INFO] epoch=2300, loss=0.5007938
[INFO] epoch=2400, loss=0.5007938
[INFO] epoch=2500, loss=0.5007938
[INFO] epoch=2600, loss=0.5007938
[INFO] epoch=2700, loss=0.5007938
[INFO] epoch=2800, loss=0.5007938
[INFO] epoch=2900, loss=0.

In [8]:
# now that our network is trained, loop over the XOR data points
for (x, target) in zip(X, y):
	# make a prediction on the data point and display the result
	# to our console
	pred = nn.predict(x)[0][0]
	step = 1 if pred > 0.5 else 0
	print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(
		x, target[0], pred, step))

[INFO] data=[0 0], ground-truth=0, pred=0.5161, step=1
[INFO] data=[0 1], ground-truth=1, pred=0.5000, step=0
[INFO] data=[1 0], ground-truth=1, pred=0.4839, step=0
[INFO] data=[1 1], ground-truth=0, pred=0.4678, step=0


### Backpropagation with Python Example: MNIST Sample

In [9]:
# load the MNIST dataset and apply min/max scaling to scale the
# pixel intensity values to the range [0, 1] (each image is
# represented by an 8 x 8 = 64-dim feature vector)
print("[INFO] loading MNIST (sample) dataset...")
digits = datasets.load_digits()
data = digits.data.astype("float")
data = (data - data.min()) / (data.max() - data.min())
print("[INFO] samples: {}, dim: {}".format(data.shape[0],
	data.shape[1]))

[INFO] loading MNIST (sample) dataset...
[INFO] samples: 1797, dim: 64


In [10]:
# construct the training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data,
	digits.target, test_size=0.25)

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [11]:
# train the network
print("[INFO] training network...")
nn = NeuralNetwork([trainX.shape[1], 32, 16, 10])
print("[INFO] {}".format(nn))
nn.fit(trainX, trainY, epochs=1000)

[INFO] training network...
[INFO] NeuralNetwork: 64-32-16-10
[INFO] epoch=1, loss=612.5098511
[INFO] epoch=100, loss=5.8360749
[INFO] epoch=200, loss=2.4952593
[INFO] epoch=300, loss=2.0198691
[INFO] epoch=400, loss=1.8451738
[INFO] epoch=500, loss=1.7558630
[INFO] epoch=600, loss=1.7020598
[INFO] epoch=700, loss=1.6662816
[INFO] epoch=800, loss=1.6408578
[INFO] epoch=900, loss=1.6219075
[INFO] epoch=1000, loss=1.6072627


In [12]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = nn.predict(testX)
predictions = predictions.argmax(axis=1)
print(classification_report(testY.argmax(axis=1), predictions))

[INFO] evaluating network...
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        45
           1       0.91      0.97      0.94        40
           2       1.00      0.97      0.98        33
           3       0.95      0.93      0.94        43
           4       1.00      1.00      1.00        51
           5       0.93      0.98      0.95        41
           6       1.00      0.98      0.99        50
           7       1.00      1.00      1.00        50
           8       0.96      0.91      0.93        47
           9       0.96      0.94      0.95        50

    accuracy                           0.97       450
   macro avg       0.97      0.97      0.97       450
weighted avg       0.97      0.97      0.97       450

